In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00


# 1. KoGPT2로 문장 생성하기


In [ ]:
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
sent = '근육이 커지기 위해서는'

In [ ]:
input_ids = tokenizer.encode(sent, return_tensors='pt')
print(input_ids)

tensor([[33245, 10114, 12748, 11357]])


In [ ]:
tokenizer.decode([33245])

'근육이'

In [ ]:
tokenizer.decode([10114])

'커'

In [ ]:
tokenizer.decode([12748])

'지기'

In [ ]:
tokenizer.decode([11357])

'위해서는'

In [ ]:
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [ ]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

# 2. Numpy로 Top 5 뽑기

In [ ]:
import numpy as np
import torch

In [ ]:
output = model(input_ids)

In [ ]:
output.logits

tensor([[[-5.2968, -5.2453, -4.7947,  ...,  0.0246, -1.1457, -0.6493],
         [-4.7908, -5.4113, -4.8760,  ..., -0.9310, -3.6518, -5.8186],
         [-4.0486, -3.5685, -5.7360,  ..., -2.4099, -2.4636, -3.5292],
         [-5.4558, -6.0249, -6.1399,  ..., -0.2245, -4.0262, -3.2469]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
output.logits.shape

torch.Size([1, 4, 51200])

In [ ]:
logits = output.logits[0, -1]

In [ ]:
logits.shape

torch.Size([51200])

In [ ]:
top5 = torch.topk(logits, k=5)
tokens = [tokenizer.decode(token_id) for token_id in top5.indices.tolist()]

In [ ]:
print(tokens)

['무엇보다', '우선', '반드시', '피부', '무엇보다도']


# 3. Numpy Top 5로 문장 생성하기

In [ ]:
import random

In [ ]:
sent = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent, return_tensors='pt')

In [ ]:
print(input_ids)

tensor([[33245, 10114, 12748, 11357, 11488,  9929,  9199, 11342,  9048,  8146,
          9582, 14564,  6903, 11887,  9177, 14564,  9153, 11506,  9278,  9564,
         36960,  9080, 14564, 13168,  9021,  9153, 12339, 13911, 13524, 15285,
          8143, 34457,  9172,  7141,  7595,   387, 10055,  9775,  9121,  6826,
           681, 19746, 42735, 26421,   387,  9172, 15439, 12178, 10325, 28805]])


In [ ]:
import random
import time

random.seed(time.time())

In [ ]:
while len(input_ids[0]) < 50:
    with torch.no_grad():
        output = model(input_ids)
    logits = output.logits[0, -1]
    top5 = torch.topk(logits, k=30)
    token_id = random.choice(top5.indices.tolist())
    input_ids = torch.cat([input_ids, torch.tensor([[token_id]])], dim=1)

In [ ]:
print(input_ids)

tensor([[33245, 10114, 12748, 11357, 11488,  9929,  9199, 11342,  9048,  8146,
          9582, 14564,  6903, 11887,  9177, 14564,  9153, 11506,  9278,  9564,
         36960,  9080, 14564, 13168,  9021,  9153, 12339, 13911, 13524, 15285,
          8143, 34457,  9172,  7141,  7595,   387, 10055,  9775,  9121,  6826,
           681, 19746, 42735, 26421,   387,  9172, 15439, 12178, 10325, 28805]])


In [ ]:
print(len(input_ids[0]))

50


In [ ]:
tokenizer.decode(input_ids[0])

'근육이 커지기 위해서는 반드시 필요한 것이 건강이다.\n이처럼 피부과 의사들은 피부 노화에 가장 많이 노출되어 있는 피부 질환으로 노화로 인하여 생기는 주름의 종류를 ‘눈밑, 팔자, 미간’ 순으로 분류한다.\n특히, ‘주름’을 가진 눈은'